# 1. Загрузка и группировка

In [434]:
!pip3 install pandas



You should consider upgrading via the '/Users/maxim.gilman/venv/bin/python -m pip install --upgrade pip' command.


In [435]:
import pandas as pd

# Чтение данных
### Сет распределен по нескольким файлам, поэтому необходимо их объединить. 

In [436]:
answers1 = pd.read_csv('S08_question_answer_pairs.txt', sep='\t')
answers2 = pd.read_csv('S09_question_answer_pairs.txt', sep='\t')
answers3 = pd.read_csv('S10_question_answer_pairs.txt', sep='\t', encoding='latin-1')

answers = pd.concat([answers1, answers2, answers3])
#вывод данных - скрыт для уменьшения объема 
#answers.head(5)


### Сет содержит данные о сложности вопроса по мнению спрашивающего и отвечающего и номер статьи, из которой взят ответ - нам это не нужно. Поэтому переходим к очистке

In [437]:

answers.drop(labels=['DifficultyFromQuestioner', 'DifficultyFromAnswerer', 'ArticleFile'], axis='columns', inplace=True)

#вывод данных - скрыт для уменьшения объема 
#answers.head(20)

# Первичная обработка данных
## Обработка дубликатов и пустых значений
1. Сет содержит вопросы без ответа
2. Сет содержит повторяющиеся вопросы

#### Дубликаты неоходимо удалить - они не несут дополнительной смысловой нагрузки, но увеличивают объем для обработки

#### Сначала удаляем вопросы без ответа, затем повторяющиеся вопросы

In [438]:
print('Наличие null:', answers.isnull().values.any())
print('Наличие NaN:', answers.isna().values.any())

Наличие null: True
Наличие NaN: True


In [439]:
#вывод данных - скрыт для уменьшения объема 
#answers.isnull().head(5)

In [440]:
print('Размер начального сета', answers.size)

#предполагается, что нам необходимо заполнение всех полей
not_null_answers = answers.dropna(axis='rows', how='any')
print('Размер сета без пустых значений', not_null_answers.size)

# при удалении дупликатов оставляем только первое вхождение
no_duplicates_answers = not_null_answers.drop_duplicates(subset='Question', keep='first')
print('Размер сета без дубликатов', no_duplicates_answers.size)
#вывод данных - скрыт для уменьшения объема 
#no_duplicates_answers.head(20)

Размер начального сета 11991
Размер сета без пустых значений 10263
Размер сета без дубликатов 6606


In [441]:
print('Наличие null:', no_duplicates_answers.isnull().values.any())
print('Наличие NaN:', no_duplicates_answers.isna().values.any())

Наличие null: False
Наличие NaN: False


In [442]:
no_duplicates_answers.loc[no_duplicates_answers['Question'].str.contains("lincoln")]

,ArticleTitle,Question,Answer
37,Abraham_Lincoln,Do scholars rank lincoln among the top three p...,Yes
39,Abraham_Lincoln,Did lincoln have 18 months of schooling?,Yes
45,Abraham_Lincoln,When was the first photgraph of lincoln taken?,1846


In [443]:
exp = no_duplicates_answers.copy()
exp['lower_question'] = ''

for index, row in exp.iterrows():
    row['lower_question'] = row['Question'].lower()

#вывод данных - скрыт для уменьшения объема 
#exp.head(5)

In [444]:
#вывод данных - скрыт для уменьшения объема 
#exp.loc[exp['Question'].str.contains("lincoln", case=False)]

# 2. Стоп-слова, очистка

# Векторизация из коробки. Пример на русском


In [445]:
from sklearn.feature_extraction.text import  CountVectorizer

text_example = ['Мама мыла раму','Мама использовала мыло', 'Мыло мыло руки мамы']
bag_of_words = CountVectorizer().fit_transform(text_example)
#вывод данных - скрыт для уменьшения объема 
#bag_of_words.toarray()

# Базовые модели оценки похожести и начало очистки
### Сначала вспомним данные, которые мы получили на прошлом шаге 
(если начинаете проект не с начальной страницы - нужно выполнить все операции с предыдущих)

In [446]:
#вывод данных - скрыт для уменьшения объема 
#exp.head(5)

In [447]:
# установка пакета с графиками дополнительно к станадартным библиотекам. 
!pip install chart_studio



You should consider upgrading via the '/Users/maxim.gilman/venv/bin/python -m pip install --upgrade pip' command.


In [448]:
import re
from collections import Counter
import plotly
import plotly.graph_objs as go

# Разделение на токены - отдельные слова

In [449]:
text = ' '.join(exp['lower_question'])
tokens = re.findall(r"[\w']+", text.lower())

#вывод данных - скрыт для уменьшения объема 
#print('\nTokens:\n\n', tokens[:100])

In [450]:
term_freq_dict = Counter(tokens)
term_freq_tuple = [(key, value) for key, value in term_freq_dict.items()]

sorted_count_words =sorted(term_freq_tuple, key=lambda x: x[1], reverse=True)

#вывод данных - скрыт для уменьшения объема 
#sorted_count_words[:50]

In [451]:

words = list(map(lambda x: x[0], sorted_count_words)) 
count = list(map(lambda x: x[1], sorted_count_words)) 
dict_count = dict(zip(words, count))  # .ToDict(x=>x[0], x=>x[1])

plotData = [go.Bar(x=list(dict_count.keys()), y=list(dict_count.values()))]
plotData

[Bar({
     'x': [the, of, is, ..., plains, crossed, grevy's],
     'y': [1332, 788, 636, ..., 1, 1, 1]
 })]

In [452]:
# Графовое представление 
#вывод данных - скрыт для уменьшения объема 
#plotly.offline.iplot(plotData,)

In [453]:
popularPlotData = [go.Bar(x=list(dict_count.keys()), y=list(filter(lambda f: f>50,dict_count.values())))]

#вывод данных - скрыт для уменьшения объема 
#plotly.offline.iplot(popularPlotData,)

# Обработка притяжательных местоимений
- В сете содержатся указания на одушевленный предмет статьи **-his/her, he/she #####** (напр. *Was his (Alessandro_Volta) 1800 paper written in French ?*)
- В сете содержатся указания на неодушевленный предмет статьи **-this #####** (напр. *What connected the Akans to this (Ghana) Empire?*)
  
Понять о чем (ЭТОМ) говорится можно из контекста и названия темы. Но нам важно сохранить объект предложения внутри самого вопроса.

Предлагается заменить местоимение на конкретное слово - тему вопроса, однако только в тех случаях, когда тема не содержится в вопросе и это местоимение является явным на неё указателем:

(напр. "When did Lincoln begin his political career?"). В данном случае замена не производится (а his удалится на следующем шаге при очистке стоп-слов).

In [454]:
exp_without_links = exp.copy()
exp_without_links['Theme column'] = ''

links = ['his','her','its','their', 'he', 'she', 'it', 'this', 'that', 'these', 'those']


for index, row in exp_without_links.iterrows():
    # Берем объект из заголовка вопроса и заменяем его на строку с пробелами
    row['ArticleTitle'] = row['ArticleTitle'].replace('_', ' ').lower()
    
    title_words = list(map(lambda x: x.lower(),row['ArticleTitle'].split()))
    lowerQuestion = row['lower_question'].split()

    has_title_in_question = any(set.intersection(set(title_words), set(lowerQuestion)))
    has_link_in_question = any(set.intersection(set(links), set(lowerQuestion)))

    if not has_title_in_question and has_link_in_question:      
        s = row['Question'].lower()
        row['Theme column'] = re.sub(r'( he )|( his )|( her )|( its )|( their )|( she )|( this )|( that )|( these )|( those )]', ' '+ row['ArticleTitle'] + ' ', ' ' + s + ' ').strip()
    else:
         row['Theme column'] = row['lower_question']
#вывод данных - скрыт для уменьшения объема 
#exp_without_links.head(5)

## Много артиклей, глаголов to be, притяжательных местоимений.
### Большинство слов в топ 50 по выборке смысла не несут. 
То есть по смысловой нагрузке **"Lincoln was a president"**  и **"Lincoln president"** примерно идентичны. Как минимум добавляя и или удаляя лишние слова мы узнаем только время (настоящее, прошедшее, будущее) когда Линьколн президент

In [455]:
!pip install spacy
!python -m spacy download en_core_web_sm


You should consider upgrading via the '/Users/maxim.gilman/venv/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 13.9 MB 278 kB/s eta 0:00:01


You should consider upgrading via the '/Users/maxim.gilman/venv/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [456]:
import numpy as np
import spacy

nlp = spacy.load('en_core_web_sm')

In [457]:
stop_words = nlp.Defaults.stop_words
print('\nИзначальный набор\n',len(stop_words))
#вывод данных - скрыт для уменьшения объема 
#list(stop_words)[:50]


Изначальный набор
 322


In [458]:
#добавление отловленных слов из списка токенизированных, кроме when/where, language 

stop_words_changed = stop_words
stop_words_changed|= {'the','of','is','a', 'was','to','are','and','do','for','does','have','as',
                 'an','are','many','he','she','it','on','his','her','by','with','that','this','most','at','there','short','long',
                '\'s','\\\'s','.','?','!',',','&', '\?'}
#просим не учитыать слова, которые являются значимыми для нас.  
#удаляем только одну форму - become (т.к. позже будем лемматизировать)
stop_words_changed-= {'amount', 'another', 'anything', 'become',
'what',
'when',
'where',
'whereafter',
'which',
'who',
'whom',
'whose',
'why',}
# вопросительные слова мы оставляем!


print('\nИзмененный набор: ',len(stop_words_changed))
#вывод данных - скрыт для уменьшения объема 
#list(stop_words_changed)[:10]


Измененный набор:  322


In [459]:
for index, row in exp_without_links.iterrows():
    new_row = ''
    for word in  row['Theme column'].replace('\'',' \'').split():
        if word not in stop_words_changed:
            new_row += ' '+word
    row['Theme column']=new_row.replace('?','')
#вывод данных - скрыт для уменьшения объема 
#exp_without_links.head(50)

# 3. TF-IDF, WB

# Итоги прошлой встречи

- очистка данных от стоп-слов
- избавление от различий в регистре
  

In [460]:
exp_without_links.head(5)

,ArticleTitle,Question,Answer,lower_question,Theme column
0,abraham lincoln,Was Abraham Lincoln the sixteenth President of...,yes,was abraham lincoln the sixteenth president of...,abraham lincoln sixteenth president united st...
2,abraham lincoln,Did Lincoln sign the National Banking Act of 1...,yes,did lincoln sign the national banking act of 1...,lincoln sign national banking act 1863
4,abraham lincoln,Did his mother die of pneumonia?,no,did his mother die of pneumonia?,abraham lincoln mother die pneumonia
6,abraham lincoln,How many long was Lincoln's formal education?,18 months,how many long was lincoln's formal education?,lincoln formal education
8,abraham lincoln,When did Lincoln begin his political career?,1832,when did lincoln begin his political career?,when lincoln begin political career


# Окончание подготовки данных

## Нормальное слово

In [461]:
testNlp = nlp("I wasn't at the Simon's party last year. He became nervous when I saw him")
infinitives = [token.lemma_ for token in testNlp]
#print(' '.join(infinitives))
print(infinitives)

['I', 'be', 'not', 'at', 'the', 'Simon', "'s", 'party', 'last', 'year', '.', 'he', 'become', 'nervous', 'when', 'I', 'see', 'he']


In [462]:
resultColumnName = 'Resulted prepared data'
exp_without_links[resultColumnName] = ''




In [463]:
for index, row in exp_without_links.iterrows():
    simlplifiedResult = nlp(row['Theme column'])
    infinitives = [token.lemma_ for token in simlplifiedResult]
    row[resultColumnName] = ' '.join(infinitives).strip()

In [464]:
exp_without_links.head(5)

,ArticleTitle,Question,Answer,lower_question,Theme column,Resulted prepared data
0,abraham lincoln,Was Abraham Lincoln the sixteenth President of...,yes,was abraham lincoln the sixteenth president of...,abraham lincoln sixteenth president united st...,abraham lincoln sixteenth president united states
2,abraham lincoln,Did Lincoln sign the National Banking Act of 1...,yes,did lincoln sign the national banking act of 1...,lincoln sign national banking act 1863,lincoln sign national banking act 1863
4,abraham lincoln,Did his mother die of pneumonia?,no,did his mother die of pneumonia?,abraham lincoln mother die pneumonia,abraham lincoln mother die pneumonia
6,abraham lincoln,How many long was Lincoln's formal education?,18 months,how many long was lincoln's formal education?,lincoln formal education,lincoln formal education
8,abraham lincoln,When did Lincoln begin his political career?,1832,when did lincoln begin his political career?,when lincoln begin political career,when lincoln begin political career


# Мешок слов
## Повторим вычисления для токенов (т.к. даннные поменялись)

In [465]:
!pip3 install scipy
!pip3 install sklearn


You should consider upgrading via the '/Users/maxim.gilman/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/maxim.gilman/venv/bin/python -m pip install --upgrade pip' command.


In [466]:
text = ' '.join(exp_without_links[resultColumnName])
tokens = list(set(re.findall(r"[\w']+", text.lower())))
print('\nTokens:\n\n', tokens[:10])

print(len(tokens))


Tokens:

 ['commonly', 'circulate', 'order', 'entrepreneur', 'canada', 'use', 'piter', 'garden', 'leathery', 'reservoir']
3485


In [467]:
full_tokens = set(tokens)
vectors = [[sentence.count(token) for token in tokens]
           for sentence in tokens]
#вывод данных - скрыт для уменьшения объема 
#print([(i, word) for i, word in enumerate(full_tokens)])
#vectors

In [468]:
import scipy.spatial

test_sentence = vectors[1] #choose any

#вывод данных - скрыт для уменьшения объема 
#print(test_sentence)


    

In [469]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = list('|'.join(text.split()).split('|'))

vectorizer = CountVectorizer()
#вывод данных - скрыт для уменьшения объема 
#print( vectorizer.fit_transform(corpus).todense() )
#print( vectorizer.vocabulary_ )

# TF, IDF, TF-IDF


## TF - частота слова
## IDF - обратная частота документа
## TF-IDF - произведение TF * IDF

### Подготавливаем методы для расчета

In [470]:
def compute_tf(text):
    #Считаем частотность всех терминов во входном массиве с помощью 
    #метода Counter библиотеки collections, но уже в очищенной Theme column
    tf_text = collections.Counter(text)
    for i in tf_text:
        tf_text[i] = tf_text[i]/float(len(text))
    return tf_text
def compute_idf(word, corpus):
#на вход берется слово, для которого считаем IDF
#и корпус документов в виде списка списков слов
        #количество документов, где встречается искомый термин
        #считается как генератор списков
        return math.log10(len(corpus)/sum([1.0 for i in corpus if word in i]))
    
def compute_idf_another(word, corpus):
    data = [Counter(i) for i in corpus if word in i]
    final_counter = Counter()
    for i in data:
        final_counter += i
    most_common_word = final_counter.most_common(1)[0][1]
    return math.log10(1 + (most_common_word/float(sum([1 for i in corpus if word in i]))))

def compute_tfidf(corpus):

    documents_list = []
    for text in corpus:
        tf_idf_dictionary = {}
        computed_tf = compute_tf(text)
        for word in computed_tf:
            tf_idf_dictionary[word] = computed_tf[word] * compute_idf(word, corpus)
        documents_list.append(tf_idf_dictionary)
    return documents_list

def compute_tfidf_another(corpus):

    documents_list = []
    for text in corpus:
        tf_idf_dictionary = {}
        computed_tf = compute_tf(text)
        for word in computed_tf:
            tf_idf_dictionary[word] = computed_tf[word] * compute_idf_another(word, corpus)
        documents_list.append(tf_idf_dictionary)
    return documents_list

In [471]:
from collections import Counter
import collections

text = exp_without_links[resultColumnName]
text_separated_by_word = list(map(lambda x: x.split(),'|'.join(text).split('|')))
tf=compute_tf(text)
#вывод данных - скрыт для уменьшения объема 
#print(tf)
#text_separated_by_word

In [472]:
import math
print(compute_idf('president',text_separated_by_word))
print(compute_idf_another('president',text_separated_by_word))

1.7987492702854573
0.3132644520809928


In [473]:
tfidf=compute_tfidf(text_separated_by_word)

print(tfidf)

[{'abraham': 0.4406412183832856, 'lincoln': 0.32130732777748583, 'sixteenth': 0.5571362191059555, 'president': 0.2997915450475762, 'united': 0.35206139887624316, 'states': 0.36112100926334195}, {'lincoln': 0.32130732777748583, 'sign': 0.4406412183832856, 'national': 0.3440106189471506, 'banking': 0.5571362191059555, 'act': 0.4406412183832856, '1863': 0.5571362191059555}, {'abraham': 0.5287694620599428, 'lincoln': 0.385568793332983, 'mother': 0.4602849258955016, 'die': 0.3622676795186956, 'pneumonia': 0.6685634629271466}, {'lincoln': 0.6426146555549717, 'formal': 0.9552320199720234, 'education': 0.8548886880840297}, {'when': 0.24918146032553531, 'lincoln': 0.385568793332983, 'begin': 0.45272721371762165, 'political': 0.4777149610392816, 'career': 0.5731392119832142}, {'what': 0.09962375384227924, 'legal': 0.45679288721796174, 'tender': 0.5571362191059555, 'act': 0.4406412183832856, '1862': 0.5571362191059555, 'establish': 0.39046955243928877}, {'who': 0.2618787118297567, 'suggest': 0.57

In [474]:
tfidf_another=compute_tfidf_another(text_separated_by_word)
print(tfidf_another)

[{'abraham': 0.050171665943996864, 'lincoln': 0.050171665943996864, 'sixteenth': 0.050171665943996864, 'president': 0.0522107420134988, 'united': 0.050171665943996864, 'states': 0.050171665943996864}, {'lincoln': 0.050171665943996864, 'sign': 0.050171665943996864, 'national': 0.050171665943996864, 'banking': 0.050171665943996864, 'act': 0.050171665943996864, '1863': 0.050171665943996864}, {'abraham': 0.06020599913279624, 'lincoln': 0.06020599913279624, 'mother': 0.06020599913279624, 'die': 0.06020599913279624, 'pneumonia': 0.06020599913279624}, {'lincoln': 0.10034333188799373, 'formal': 0.10034333188799373, 'education': 0.10034333188799373}, {'when': 0.06020599913279624, 'lincoln': 0.06020599913279624, 'begin': 0.06020599913279624, 'political': 0.06020599913279624, 'career': 0.06020599913279624}, {'what': 0.050171665943996864, 'legal': 0.050171665943996864, 'tender': 0.050171665943996864, 'act': 0.050171665943996864, '1862': 0.050171665943996864, 'establish': 0.050171665943996864}, {'w

# Косинусное сходство

In [475]:
#немного переделенные методы для tf-idf 
import operator


def tokenize(doc):
    words = [word.replace(',', '').lower() for word in doc.split()]
    return words


def build_terms(corpus):
    terms = {}
    current_index = 0
    for doc in corpus:
        for word in tokenize(doc):
            if word not in terms:
                terms[word] = current_index
                current_index += 1
    return terms


def tf(document, terms):
    words = tokenize(document)
    total_words = len(words)
    doc_counter = Counter(words)
    for word in doc_counter:
        # Можно и не делить, а оставить как есть, с частотой
        doc_counter[word] /= total_words
    tfs = [0 for _ in range(len(terms))]
    for term, index in terms.items():
        tfs[index] = doc_counter[term]
    return tfs


def _count_docs_with_word(word, docs):
    counter = 1
    for doc in docs:
        if word in doc:
            counter += 1
    return counter


def idf(documents, terms):
    idfs = [0 for _ in range(len(terms))]
    total_docs = len(documents)
    for word, index in terms.items():
        docs_with_word = _count_docs_with_word(word, documents)
        idf = 1 + math.log10(total_docs / docs_with_word)
        idfs[index] = idf
    return idfs


def _merge_td_idf(tf, idf, terms):
    return [tf[i] * idf[i] for i in range(len(terms))]


def build_tfidf(corpus, document, terms):
    doc_tf = tf(document, terms)
    doc_idf = idf(corpus, terms)
    return _merge_td_idf(doc_tf, doc_idf, terms)


def cosine_similarity(vec1, vec2):
    def dot_product2(v1, v2):
        return sum(map(operator.mul, v1, v2))

    def vector_cos5(v1, v2):
        prod = dot_product2(v1, v2)
        len1 = math.sqrt(dot_product2(v1, v1))
        len2 = math.sqrt(dot_product2(v2, v2))
        return prod / (len1 * len2)
         

    return vector_cos5(vec1, vec2)






In [ ]:
tf_idf_total = []
corpus = tuple(text)#[:200] # общее время расчета - 23m 11s # [:1000]
terms = build_terms(corpus)

for document in corpus:
    tf_idf_total.append(build_tfidf(corpus, document, terms))

#for doc_rating in tf_idf_total:
#    print(doc_rating)

In [ ]:
print(terms.keys())
query = 'lincoln president united states'
print("QUERY:",query )
query_tfidf = build_tfidf(corpus, query, terms)
for index, document in enumerate(tf_idf_total):
    print("Similarity with DOC", index, "=", cosine_similarity(query_tfidf, document))

In [ ]:
exp_without_links.head(10)

# 4. Расстояния, шинглы

# Ливенштейн

In [ ]:
!pip install fuzzywuzzy
!pip3 install python-Levenshtein

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
# мыым -> мымы -> мама

exmpl = 'мама' #убрать пробелы
original = 'мамы'
print(exmpl)
print(fuzz.token_sort_ratio(exmpl, original))
print(fuzz.token_set_ratio(exmpl,  original))
print(fuzz.token_set_ratio(exmpl,  'мыло'))

In [ ]:
exmpl = exp_without_links.iloc[0][resultColumnName]
test = 'lincoln sixteenth president'
print('Example: ' + exmpl)
print('Test: ' + test)
print(fuzz.token_sort_ratio(exmpl, test))
print(fuzz.token_set_ratio(exmpl,  test))

In [ ]:
error_test = 'abrahma linconl sixteneth persident unitde statse'

print(fuzz.token_sort_ratio(test,  error_test))

# Алгоритм Шинглов — поиск нечетких дубликатов текста

In [ ]:
def genshingle(source, shingle_len):
    import binascii
    #длина шингла - 3--5--7
    out = [] 
    for i in range(len(source)-(shingle_len-1)): #делим на шинглы
        out.append (binascii.crc32(' '.join( [x for x in source[i:i+shingle_len]] ).encode('utf-8')))
    #print(out)
    return out

def compare(source1,source2):
    same = 0
    for i in range(len(source1)):
        if source1[i] in source2:
            same = same + 1

    return same*2/float(len(source1) + len(source2))*100

In [ ]:
text1 = exp_without_links.iloc[0][resultColumnName] # Текст 1 для сравнения - abraham lincoln sixteenth president united states
text2 = 'lincoln sixteenth president' # Текст 2 для сравнения - обработанный
text3 = 'was abraham linсoln the 16\'th president of Ameriсa' # Текст 3 для сравнения -не очищенный
text4 = 'barak obama' # Текст 4 - совсем другой
text0 = text1 # Текст 5 - копия точная

print('Длина шингла - 3')
cmp0 = genshingle(text0,3)
cmp1 = genshingle(text1,3)
cmp2 = genshingle(text2,3)
cmp3 = genshingle(text3,3)
cmp4 = genshingle(text4,3)
print ('\n Original vs. Original '+str(compare(cmp1,cmp0)))
print ('\n Original vs. clear. pretty near '+str(compare(cmp1,cmp2)))
print ('\n Original vs. fact value. Not pretty near '+str(compare(cmp1,cmp3)))
print ('\n Original vs. other. Not near almost '+str(compare(cmp1,cmp4)))
print('-------------------------------')



print('Длина шингла - 5')
cmp0 = genshingle(text0,5)
cmp1 = genshingle(text1,5)
cmp2 = genshingle(text2,5)
cmp3 = genshingle(text3,5)
cmp4 = genshingle(text4,5)
print ('\n Original vs. clear. pretty near '+str(compare(cmp1,cmp2)))
print ('\n Original vs. fact value. Not pretty near '+str(compare(cmp1,cmp3)))
print('-------------------------------')


print('Длина шингла - 7')
cmp0 = genshingle(text0,7)
cmp1 = genshingle(text1,7)
cmp2 = genshingle(text2,7)
cmp3 = genshingle(text3,7)
cmp4 = genshingle(text4,7)
print ('\n Original vs. clear. pretty near '+str(compare(cmp1,cmp2)))
print ('\n Original vs. fact value. Not pretty near '+str(compare(cmp1,cmp3)))

### Поиск LCS. Difflib

In [ ]:
import difflib

def similarity(s1, s2):
    normalized1 = s1.lower()
    normalized2 = s2.lower()
    matcher = difflib.SequenceMatcher(None, normalized1, normalized2)
    return matcher.ratio()

# Демо-поиск по тексту

In [ ]:
exp_without_links.head(5)

In [ ]:
input_example = 'Did the election of 1880 was won by Lincoln?'

### Очистка стоп-слов входной строки

In [ ]:
clear_input_exmpl=''
new_row = ''
question = ''
for word in  input_example.split():
    if word not in stop_words:
        #print(word)
        new_row += ' '+word
        clear_input_exmpl=new_row.strip() .lower().replace('?','')

print(clear_input_exmpl) 

### Нормальные формы слова и настоящее время

In [ ]:
base_form =  nlp(clear_input_exmpl)

infinitives = [token.lemma_ for token in base_form]

base_form_clear_input_example = ' '.join(infinitives).strip()
base_form_clear_input_example

## Подбор адекватных значений модели

In [ ]:
# Пробелы убраны!

base_form_clear_input_example = base_form_clear_input_example.replace(' ','')
levin_counter_set=50
real_levin_counter=levin_counter_set
levin_question=''

for question in text:
    question_with_no_space = question.replace(' ','')
    if(fuzz.token_set_ratio(question_with_no_space, base_form_clear_input_example)>=real_levin_counter):
        real_levin_counter=fuzz.token_set_ratio(question_with_no_space, clear_input_exmpl)
        levin_question= question
        real_levin_counter = fuzz.token_set_ratio(question_with_no_space, base_form_clear_input_example)

print(base_form_clear_input_example)
print('it is most similar to:')
print(levin_question)
print('Levenshtein distance was to be:')
print(levin_counter_set)
print('but found:')
print(real_levin_counter)
print('Quesion And Answer are:')
exp_without_links.loc[exp_without_links[resultColumnName] == levin_question, ['Question','Answer']]

# Русская модель - бери и делай

### [Natasha_Lib. https://github.com/natasha/natasha](https://github.com/natasha/natasha)

# 5. Проверка результатов и адекватности

### Загрузка тестовых вопросов

In [ ]:
test_answers_1 = pd.read_csv('test_questions.csv', sep=';', encoding='latin-1')

test_answers = pd.concat([test_answers_1])

test_exp=test_answers.copy()

test_exp.head(5)

### Очистка тестовых данных


In [ ]:
test_exp['Result'] = ''
links = ['his','her','its','their', 'he', 'she', 'it', 'this', 'that', 'these', 'those']

for index, row  in test_exp.iterrows():
    clear_input_exmpl=''
    new_row = ''
    old_row = row['Question'].lower()
    
    # Удаление местоимений
    row['ArticleTitle'] = row['ArticleTitle'].replace('_', ' ').lower()
    title_words = list(map(lambda x: x.lower(),row['ArticleTitle'].split()))

    has_title_in_question = any(set.intersection(set(title_words), set(old_row)))
    has_link_in_question = any(set.intersection(set(links), set(old_row)))

    if not has_title_in_question and has_link_in_question:      
        old_row = re.sub(r'( he )|( his )|( her )|( its )|( their )|( she )|( this )|( that )|( these )|( those )]', ' '+ row['ArticleTitle'] + ' ', ' ' + old_row + ' ').strip()
    else:
        old_row = old_row
    
    # Нормализация слова    
    base_form =  nlp(old_row)
    infinitives = [token.lemma_ for token in base_form]
    base_form_clear_input_example = ' '.join(infinitives).strip()
    #print(base_form_clear_input_example)
    
    # Удаление стоп-слов
    for word in  base_form_clear_input_example.split():
        if word not in stop_words:
            new_row += ' '+word 
    row['Result']=new_row.strip().lower().replace('?','').replace('bear','born')




In [ ]:
test_exp.head(10)

### Косинусное расстояние

In [ ]:
test_exp['cosine_question'] = '' #предикт вопроса по cosine distance
test_exp['cosine_answer'] = ''
test_count = test_exp['Answer'].count()

for index, row  in test_exp.iterrows():
    max_similarity = 0.0
    similar_index=0

    query_tfidf = build_tfidf(corpus, row['Result'], terms)
    for index_tf_idf, document in enumerate(tf_idf_total):
        current_similarity = cosine_similarity(query_tfidf, document)
        if(max_similarity < current_similarity):
            max_similarity = current_similarity
            similar_index = index_tf_idf
        #print("Similarity with DOC", index, "=", cosine_similarity(query_tfidf, document))
    #print(exp_without_links.iloc[similar_index][resultColumnName])
    #print(row['Result'])
    row['cosine_question'] = exp_without_links.iloc[similar_index]['Question'].replace('[\'','').replace('\']','').replace('[\"','').replace('\"]','').lower().strip()
    row['cosine_answer'] = exp_without_links.iloc[similar_index]['Answer'].replace('[\'','').replace('\']','').replace('[\"','').replace('\"]','').lower().strip()
    print(f'{index} / {test_count}')
    


### Предрасчет тестов для Левенштейна и Шинглов

In [ ]:
test_exp['Levin_question'] = '' #предикт вопроса по левенштейну
test_exp['Levin_answer'] = ''
test_exp['Shingles_question'] = '' #предикт вопроса по шинглам
test_exp['Shingles_answer'] = ''

for index, row in test_exp.iterrows():
    
    #levin
    levin_counter=0
    levin_question=''
    for question in text:
        if(fuzz.token_set_ratio(question, row['Result'])>=levin_counter):
            levin_counter=fuzz.token_set_ratio(question, row['Result'])
            levin_question= question
    row['Levin_question'] = str(exp_without_links.loc[exp_without_links[resultColumnName] == levin_question, ['Question']].values[0]).replace('[\'','').replace('\']','').replace('[\"','').replace('\"]','').lower().strip()
    row['Levin_answer'] = str(exp_without_links.loc[exp_without_links[resultColumnName] == levin_question, ['Answer']].values[0]).replace('[\'','').replace('\']','').replace('[\"','').replace('\"]','').lower().strip()
    
    # shingle
    shingle_counter=0
    shingle_question=''
    main_cmp = genshingle(row['Result'], 3)
    for question in text:
        tmp_cmp = genshingle(question, 3)
        similatrity=compare(main_cmp,tmp_cmp)
        if(similatrity>shingle_counter):
            shingle_counter=similatrity
            shingle_question=question
    #print(shingle_question)      
    row['Shingles_question'] = str(exp_without_links.loc[exp_without_links[resultColumnName] == shingle_question, ['Question']].values[0]).replace('[\'','').replace('\']','').replace('[\"','').replace('\"]','').lower().strip()
    row['Shingles_answer'] = str(exp_without_links.loc[exp_without_links[resultColumnName] == shingle_question, ['Answer']].values[0]).replace('[\'','').replace('\']','').replace('[\"','').replace('\"]','').lower().strip()

In [ ]:
test_exp.head(10)

### Предрасчет тестов для LCS difflib

In [ ]:
test_exp['difflib_question'] = '' #предикт вопроса по difflib
test_exp['difflib_answer'] = ''
for index, row in test_exp.iterrows():
    difflib_question=''
    difflib_counter=0
    for question in text:
        if(similarity(question, row['Result'])>=difflib_counter):
            difflib_counter=similarity(question, row['Result'])
            difflib_question= question
    row['difflib_question'] = str(exp_without_links.loc[exp_without_links[resultColumnName] == difflib_question, ['Question']].values[0]).replace('[\'','').replace('\']','').replace('[\"','').replace('\"]','').lower().strip()
    row['difflib_answer'] = str(exp_without_links.loc[exp_without_links[resultColumnName] == difflib_question, ['Answer']].values[0]).replace('[\'','').replace('\']','').replace('[\"','').replace('\"]','').lower().strip()

In [ ]:
test_exp.head(10)

### Для каждого способа проверяем - совпадает ли указанный им ответ с нужным нам

In [ ]:
# 0 если ответы не сошлись, 1 если сошлись
test_exp['Levin_count'] = ''
test_exp['Shingle_count'] = ''
test_exp['difflib_count'] = ''
test_exp['cosine_count'] = ''

test_exp['any_correct']=''
test_exp['most_correct']=''


for index, row in test_exp.iterrows():
    clear_answer = row['Answer'].lower().replace(' ','')
    if(row['Levin_answer'].replace(' ','')==clear_answer):
        row['Levin_count']=1
    else:
        row['Levin_count']=0

    if(row['Shingles_answer'].replace(' ','')==clear_answer):
        row['Shingle_count']=1
    else:
        row['Shingle_count']=0
    
    if(row['difflib_answer'].replace(' ','')==clear_answer):
        row['difflib_count']=1
    else:
        row['difflib_count']=0
        
        
    if(row['cosine_answer'].replace(' ','')==clear_answer):
        row['cosine_count']=1
    else:
        row['cosine_count']=0
        
    any_correct = row['difflib_count']==1 or row['Levin_count']==1 or row['Shingle_count']==1  or row['cosine_count']==1 
    if(any_correct):
        row['any_correct']=1
    else:
        row['any_correct']=0

In [ ]:
test_exp.head(13)

In [ ]:
print('Cosine точность')
test_exp['cosine_count'].sum()/test_exp['cosine_count'].count()

In [ ]:
print('Левенштейн точность')
test_exp['Levin_count'].sum()/test_exp['Levin_count'].count()

In [ ]:
print('Шинглы точность')
test_exp['Shingle_count'].sum()/test_exp['Shingle_count'].count()

In [ ]:
print('difflib точность')
test_exp['difflib_count'].sum()/test_exp['difflib_count'].count()

In [ ]:
print('total точность')
test_exp['any_correct'].sum()/test_exp['any_correct'].count()

In [ ]:
trace1 = go.Bar(
    x=['Косинусы', 'Шинглы', 'Левенштейн', 'difflib', 'Общий'],
    y=[test_exp['cosine_count'].sum(), 
       test_exp['Shingle_count'].sum(), 
       test_exp['Levin_count'].sum(), 
       test_exp['difflib_count'].sum(), 
       test_exp['any_correct'].sum()],
    name='Успех'
)
trace2 = go.Bar(
    x=['Косинусы','Шинглы', 'Левенштейн', 'difflib', 'Общий'],
    y=[test_exp['cosine_count'].count()-test_exp['cosine_count'].sum(),
       test_exp['Levin_count'].count()-test_exp['Shingle_count'].sum(),
       test_exp['Shingle_count'].count()- test_exp['Levin_count'].sum(),
       test_exp['difflib_count'].count()-test_exp['difflib_count'].sum(),
       test_exp['any_correct'].count()-test_exp['any_correct'].sum()],
    name='Ошибка'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='grouped-bar')

In [ ]:
trace1 = go.Bar(
    x=['Косинусы', 'Шинглы', 'Левенштейн', 'difflib', 'Общий'],
    y=[test_exp['cosine_count'].sum()/test_exp['cosine_count'].count(), 
      test_exp['Shingle_count'].sum()/test_exp['Shingle_count'].count(),
      test_exp['Levin_count'].sum()/test_exp['Levin_count'].count(),
      test_exp['difflib_count'].sum()/test_exp['difflib_count'].count(),
      test_exp['any_correct'].sum()/test_exp['any_correct'].count()],
    name='Успех'
)
trace2 = go.Bar(
    x=['Косинусы','Шинглы', 'Левенштейн', 'difflib', 'Общий'],
    y=[(test_exp['cosine_count'].count()-test_exp['cosine_count'].sum())/test_exp['cosine_count'].count(),
      (test_exp['Shingle_count'].count()-test_exp['Shingle_count'].sum())/test_exp['Shingle_count'].count(),
      (test_exp['Levin_count'].count()-test_exp['Levin_count'].sum())/test_exp['Levin_count'].count(),
      (test_exp['difflib_count'].count()-test_exp['difflib_count'].sum())/test_exp['difflib_count'].count(),
      (test_exp['any_correct'].count()-test_exp['any_correct'].sum())/test_exp['any_correct'].count(),],
    name='Ошибка'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='grouped-bar')

In [ ]:
test_exp.to_csv('result.csv', sep=';')

## А где совсем - совсем не угадали?

In [ ]:
total_error = test_exp.loc[test_exp['any_correct'] == 0, ['Question', 'difflib_question','Shingles_question','Levin_question', 'cosine_question']]

print(total_error.count())
total_error

# Тест с внешним примером

In [ ]:
input_example = 'What year was born the god?'

In [ ]:
clear_input_exmpl=''
new_row = ''
old_row = input_example.lower()
    
# Удаление местоимений      
old_row = re.sub(r'( he )|( his )|( her )|( its )|( their )|( she )|( this )|( that )|( these )|( those )]', ' ' + ' ', ' ' + old_row + ' ').strip()

    
# Нормализация слова    
base_form =  nlp(old_row)
infinitives = [token.lemma_ for token in base_form]
base_form_clear_input_example = ' '.join(infinitives).strip()
#print(base_form_clear_input_example)
    
# Удаление стоп-слов
for word in  base_form_clear_input_example.split():
    if word not in stop_words:
        new_row += ' '+word 
clear_input_exmpl=new_row.strip().lower().replace('?','').replace('bear','born')


In [ ]:
clear_input_exmpl

In [ ]:
# Пробелы убраны!

base_form_clear_input_example = base_form_clear_input_example.replace(' ','')
levin_counter_set=40
real_levin_counter=levin_counter_set
levin_question=''


# Косинусное расстояние

In [ ]:
test_predict_question_cosine = '' #предикт вопроса по cosine distance
test_predict_answer_cosine = ''

max_similarity = 0.0
similar_index=0

query_tfidf = build_tfidf(corpus, clear_input_exmpl, terms)
for index_tf_idf, document in enumerate(tf_idf_total):
    current_similarity = cosine_similarity(query_tfidf, document)
    if(max_similarity < current_similarity):
        max_similarity = current_similarity
        similar_index = index_tf_idf
#print("Similarity with DOC", index, "=", cosine_similarity(query_tfidf, document))
#print(exp_without_links.iloc[similar_index][resultColumnName])
#print(row['Result'])
test_predict_question_cosine = exp_without_links.iloc[similar_index]['Question'].replace('[\'','').replace('\']','').replace('[\"','').replace('\"]','').lower().strip()
test_predict_answer_cosine = exp_without_links.iloc[similar_index]['Answer'].replace('[\'','').replace('\']','').replace('[\"','').replace('\"]','').lower().strip()


print(input_example)
print(test_predict_question_cosine)
print(test_predict_answer_cosine)

# Левинштейн

In [ ]:
test_predict_question_levin = '' #предикт вопроса по левенштейну
test_predict_answer_levin = ''


levin_counter=0
levin_question=''
for question in text:
    if(fuzz.token_set_ratio(question, clear_input_exmpl)>=levin_counter):
        levin_counter=fuzz.token_set_ratio(question, clear_input_exmpl)
        levin_question= question
test_predict_question_levin = str(exp_without_links.loc[exp_without_links[resultColumnName] == levin_question, ['Question']].values[0]).replace('[\'','').replace('\']','').replace('[\"','').replace('\"]','').lower().strip()
test_predict_answer_levin = str(exp_without_links.loc[exp_without_links[resultColumnName] == levin_question, ['Answer']].values[0]).replace('[\'','').replace('\']','').replace('[\"','').replace('\"]','').lower().strip()


print(input_example)
print(test_predict_question_levin)
print(test_predict_answer_levin) 

# Шинглы

In [ ]:
test_predict_question_shingle = '' #предикт вопроса по Шинглам
test_predict_answer_shingle = ''

shingle_counter=0
shingle_question=''
main_cmp = genshingle(clear_input_exmpl, 3)
for question in text:
    tmp_cmp = genshingle(question, 3)
    similatrity=compare(main_cmp,tmp_cmp)
    if(similatrity>shingle_counter):
        shingle_counter=similatrity
        shingle_question=question
        
test_predict_question_shingle = str(exp_without_links.loc[exp_without_links[resultColumnName] == shingle_question, ['Question']].values[0]).replace('[\'','').replace('\']','').replace('[\"','').replace('\"]','').lower().strip()
test_predict_answer_shingle = str(exp_without_links.loc[exp_without_links[resultColumnName] == shingle_question, ['Answer']].values[0]).replace('[\'','').replace('\']','').replace('[\"','').replace('\"]','').lower().strip()

print(input_example)
print(test_predict_question_shingle)
print(test_predict_answer_shingle) 

# [Доп]. 2-3. Примеры из презентации

In [ ]:
!pip3 install pymorphy2
!pip3 install pymystem3
!pip3 install nltk

In [ ]:
from nltk.stem.snowball import SnowballStemmer 
#from nltk.stem.porter import PorterStemmer

# Стэмминг по-русски

In [ ]:
words = ['мама','мамочка','мамуля','мыла','рамочку','рамулю', 'мамы','мамочки', 'моей']
snow_stemmer = SnowballStemmer(language='russian')

for word in words:
    print(word+' -> '+snow_stemmer.stem(word))

# Стэмминг по-человечески

In [ ]:
words = ['rain','raining','faith','faithful','are','is','care','caring']
snow_stemmer = SnowballStemmer(language='english')

for word in words:
    print(word+' -> '+snow_stemmer.stem(word))

# Лемматизация

In [ ]:
import pymorphy2 # OS
import pymystem3 # над яндексом (не до конца опенсорс)

m1 = pymorphy2.MorphAnalyzer()
m2 = pymystem3.Mystem()

In [ ]:
m1.parse('мамин')

In [ ]:
m2.analyze('шел')

## Библиотеки работают по-разному

In [ ]:
print(m1.parse('Мамин'))
print(m1.parse('мыла'))
print(m1.parse('раму'))

In [ ]:
m2.analyze('Мамин')

In [ ]:
m2.analyze('стекло')

# [Доп]. Обработка определений (для Андрея Рябова)

In [ ]:
!pip3 install pymorphy2
!pip3 install pymystem3

In [ ]:
import pymorphy2 # 
import pymystem3 # 

m1 = pymorphy2.MorphAnalyzer()
m2 = pymystem3.Mystem()

In [ ]:
m1.parse('мамин')
# https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html

In [ ]:
m2.analyze('Мамин')

# притяжательное прилагательное (А) - наш потенциальный кандидат

# [Доп]. Обработка алгоритмом Шинглов для букв

In [ ]:
def genshingle_word(source, shingle_len):
    import binascii
    #длина шингла - 3--5--7
    out = [] 
    for i in range(len(source)-(shingle_len-1)): #делим на шинглы
        out.append (binascii.crc32(' '.join( [x for x in source[i:i+shingle_len]] ).encode('utf-8')))
    #print(out)
    return out

def compare_word(source1,source2):
    same = 0
    for i in range(len(source1)):
        if source1[i] in source2:
            same = same + 1

    return same*2/float(len(source1) + len(source2))*100

In [ ]:
compare_word(genshingle_word('hellomynameisjake', 3), genshingle_word('hellojakeisthedog', 3))

In [ ]:
compare(genshingle('hello my name is jake', 3), genshingle('hello jake is the dog', 3))